In [1]:
print("okie")

okie


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
PINECONE_API_KEY = "pcsk_3tVDRV_9vnx6F8411DpGEvCUgx9E7ogz3guPB8VugKZSgitCYHxzrDx13dXuSRb1LaskQC"
PINECONE_API_ENV = "us-east-1"

In [4]:
def loadpdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = loadpdf("data/")

In [6]:
#extracted_data

In [7]:
# create text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("Length of my chunk:", len(text_chunks))

Length of my chunk: 39994


In [9]:
# text_chunks

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_huggingface_embeddings()

In [13]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [25]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Set environment variable before using langchain_pinecone
os.environ["PINECONE_API_KEY"] = "pcsk_3tVDRV_9vnx6F8411DpGEvCUgx9E7ogz3guPB8VugKZSgitCYHxzrDx13dXuSRb1LaskQC"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1-aws"  # REQUIRED

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "medicalbot"

# Create index if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Now use LangChain PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name,
)

In [31]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [32]:
retrieved_docs = retriever.invoke("What is Acne?")

In [33]:
retrieved_docs

[Document(id='97750e13-584e-43fc-a113-dac209c17cc2', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'data\\Medical book.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='75f52c48-c47c-41bb-8e6e-f9d7d779a7ff', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'data\\Medical book.pdf', 'total_pages': 4505.0}, page_content='Sebaceous follicles— A structure found within the\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— An oily skin moisturizer produced by\nsebaceous glands.\nTretinoin— A drug that works by increasing the\nturnover 

In [69]:
from together import Together

# Join the retrieved documents into a single string
retrieved_text = "\n".join([doc.page_content for doc in retrieved_docs])
user_question = "What is allergies?"

# Prepare the system prompt for the model
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    f"{retrieved_text}"
)

# Step 4: Send the prompt to the LLM model using Together API
client = Together(api_key="48524a1dbffe66fd732b2d490d59c15c47ec440f48a1ed0cc7ba4cbda750be64")
response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_question}
    ]
)

# Step 5: Print the response from the model
# Adjust to access the correct attribute in the response
print("Response:", response.choices[0].message.content)

Response: I don't know what allergies are based on the provided context, as it only discusses acne and other unrelated medical terms. The context does not mention allergies.
